# IMPORTS

In [12]:
# from grobid_client.grobid_client import GrobidClient
import xml.etree.ElementTree as ET
import matplotlib.pyplot as plt
import os
import re
import sys
import gensim
from gensim import corpora, models, similarities
import pickle

import os
from stop_words import get_stop_words
from sentence_transformers import SentenceTransformer
from sklearn.metrics.pairwise import cosine_similarity
from sklearn.cluster import AgglomerativeClustering, KMeans, DBSCAN
import numpy as np
# Now let's do topic modeling using LDA
from sklearn.decomposition import LatentDirichletAllocation


# si no hace bien los imports de utilsdescomenta esta linea
parent_dir = os.path.abspath(os.path.join(os.getcwd(), ".."))
sys.path.append(parent_dir)

from utils import remove_files, get_abstract, cosine

# GET papers

In [13]:
# Directorio donde se encuentran los archivos XML
xml_dir = os.path.join(parent_dir, "output")
# papers_dir = os.path.join(parent_dir, "papers")

# remove_files(xml_dir)

# client = GrobidClient(config_path="./config.json")
# client.process("processFulltextDocument", papers_dir, output=xml_dir, consolidate_citations=True, tei_coordinates=True, n=20)

# Lista para almacenar los resúmenes
papers = {}

# Procesar cada archivo XML en el directorio
for file in os.listdir(xml_dir):
    if file.endswith(".xml"):  # Verificar que el archivo sea XML
        file_path = os.path.join(xml_dir, file)
        file_name = os.path.basename(file_path)[:-15]
        tree = ET.parse(file_path)
        root = tree.getroot()
        #verificamos si el paper esta en nuestro diccionario y si no lo esta creamos una entrada para el
        if file_name not in papers:
            papers[file_name] = {}
        #get abstract
        abstract = get_abstract(root)
        papers[file_name]['abstract'] = abstract

        #get authors
        ns = {'tei': 'http://www.tei-c.org/ns/1.0'}
        autor_loc = root.findall('.//tei:sourceDesc', ns)
        acknowledgePerson = root.findall('.//tei:author', ns)
        acknowledgeOrg = root.findall('.//tei:author', ns)
        # Obtener los nombres completos de los autores
        nombres_completos = []
        for loc in autor_loc:
            autors = loc.findall('.//tei:persName', ns)
            # Encontrar el elemento forename dentro de author
                    # Verificar si se encontraron los elementos forename y surname
            for autor in autors:
                forename_elem = autor.findall('.//tei:forename', ns)
                # Encontrar el elemento surname dentro de author
                surname_elem = autor.findall('.//tei:surname', ns)
                
                try:
                    # Obtener el texto de los elementos forename y surname
                    nombre = forename_elem[0].text
                    apellido = surname_elem[0].text
                    nombre_completo = f"{nombre} {apellido}"
                except:
                    # Si no se encontraron los elementos, asignar NA a nombre y apellido
                    # print('Missing name and/or surname. Cant get full name.')
                    pass
                nombres_completos.append(nombre_completo)
            papers[file_name]['written_by'] = nombres_completos
        
        # Obtener la fecha de publicación
        try:
            published_date = root.find('.//tei:fileDesc/tei:publicationStmt/tei:date', ns).text
            papers[file_name]['published_date'] = published_date
        except:
            pass

        # Obtener el DOI (id) del documento
        doi = root.find('.//tei:fileDesc/tei:sourceDesc/tei:biblStruct/tei:idno[@type="MD5"]', ns).text
        papers[file_name]['id'] = doi

        #autores referencias
        nombres_completosRef = []
        for autor in acknowledgePerson:
            # Encontrar el elemento forename dentro de author
            forename_elem = autor.find('.//tei:forename', ns)
            # Encontrar el elemento surname dentro de author
            surname_elem = autor.find('.//tei:surname', ns)
            
            # Verificar si se encontraron los elementos forename y surname
            try:
                # Obtener el texto de los elementos forename y surname
                nombre = forename_elem.text
                apellido = surname_elem.text
                nombre_completoRef = f"{nombre} {apellido}"
            except:
                # Si no se encontraron los elementos, asignar NA a nombre y apellido
                # print('Missing name and/or surname. Cant get full name.')
                pass
            if nombre_completoRef not in nombres_completosRef and nombre_completoRef not in nombres_completos:
                nombres_completosRef.append(nombre_completoRef)
        papers[file_name]['acknowledgePerson'] = nombres_completosRef
        
        nombres_org = []
        for org in acknowledgeOrg:
            # Encontrar el elemento forename dentro de autho
            forename_elem = org.find('.//tei:orgName[@type="institution"]', ns)

            # Verificar si se encontraron los elementos forename y surname
            try:
                # Obtener el texto de los elementos forename y surname
                nombre = forename_elem.text
                nombre_org = nombre
            except:
                nombre_org = None

                # Si no se encontraron los elementos, asignar NA a nombre y apellido
                # print('Missing name and/or surname. Cant get full name.')
            if nombre_org and nombre_org not in nombres_org:
                nombres_org.append(nombre_org)
        papers[file_name]['acknowledgeOrg'] = nombres_org


print(f'Numero de documentos: {len(papers)}')

010
11621ijccsa02
1709.01907
1802.05799
1912.00
2007.03051
208
269 An Insight into Cloud Computing Paradigm and Services
6114nsa03
9-12
907-Article Text-2692-1-10-20230720
agriengineering-03-00020
CBIR USING FEATURES DERIVED BY DEEP LEARNING
DeepLearning_Le
DINOv2 Learning Robust Visual Featu
Experimental Study of DINOv2 on Radiol
hir-22-351
How good are deep models in understanding generated images
IJISRT23AUG773
IMAGEBERT CROSS-MODAL PRE-TRAINING WITH
ISRGJAHSS1952
JDSSV_shamsudheen_hennig_fi
LEARNING SUPER-FEATURES FOR IMAGE RETRIEVAL
Learning Transferable Visual Models From Natural Language Supervis
LightGlue Local Feature Matching at Light Sp
Paper11879
Plant_Disease_Detection_and_Classification_by_Deep_LearningA_Rev
Sketch Based Image Retrieval for Architecture
tft
VISION TRANSFORMERS NEED REGISTERS
Numero de documentos: 30


In [14]:
papers

{'010': {'abstract': 'Cloud computing is fast creating a revolution in the way information technology is used and procured by organizations and by individuals. In this article, we examine what cloud computing is and the importance of this new model of computing. We then examine non-military uses of cloud computing in governments across the globe, from the Unites States to Europe and Asia. Then, we look at the resource -people and computing -issues involved in shirting to cloud computing. The author then presents his six-step "Cloud Migration Strategy" for governmental agencies to shift to cloud computing. Finally, we look "over the horizon" to the implications for public sector organizations and the information technology community as the cloud computing revolution progresses.',
  'written_by': ['David Wyld'],
  'id': '34C4144FA95CCECB28B14B8159AB4498',
  'acknowledgePerson': ['C Johnson',
   'L Ulanoff',
   'C Anderson',
   '@ Knowledge',
   'D Wyld',
   'S Ferguson',
   'J Davis',
  

In [15]:
papers_list = [paper['abstract'] for paper in papers.values()]

# SIMILARITY


## BERT

In [16]:
# USING TRANSFORMERS
threshold = 0.7
# If we want to improve the similarity and use a word embeddings approach, we may use sentence transformers. This may take a while:
sbert_model = SentenceTransformer(
    "sentence-transformers/paraphrase-multilingual-MiniLM-L12-v2"
)
sentence_embeddings = sbert_model.encode(papers_list)
for i in range(len(sentence_embeddings)):
    for j in range(i + 1, len(sentence_embeddings)):
        sim = cosine(sentence_embeddings[i], sentence_embeddings[j])
        # print(
        #     f"La similitud entre el documento {list(papers.keys())[i]} y el documento {list(papers.keys())[j]} es sim {sim}"
        # )
        if sim > threshold:
            if 'has_Similarity' not in papers[list(papers.keys())[i]]:
                papers[list(papers.keys())[i]]['has_Similarity'] = {}

            if 'has_Similarity' not in papers[list(papers.keys())[j]]:
                papers[list(papers.keys())[j]]['has_Similarity'] = {}

            papers[list(papers.keys())[i]]['has_Similarity'][list(papers.keys())[j]] = sim
            papers[list(papers.keys())[j]]['has_Similarity'][list(papers.keys())[i]] = sim

# TOPIC MODELLING

## LDA

In [17]:
import nltk
from sklearn.feature_extraction.text import CountVectorizer
# let's do a countvectorizer now. This is different from TF-IDF
# we are only creating 2 topics

from nltk.corpus import stopwords
# nltk.download('stopwords')
stop_words = set(stopwords.words('english'))

tokens = [
    [
        word
        for word in paper_info['abstract'].lower().split()
        if word.isalpha() and word not in stop_words
    ]
    for paper_info in papers.values()
]


# Filtrar las stop words

# Unir los tokens filtrados de nuevo en un texto
tokens_flatten = [' '.join(lista) for lista in tokens]


In [18]:
# let's do a countvectorizer now. This is different from TF-IDF
count_vectorizer = CountVectorizer()
X = count_vectorizer.fit_transform(tokens_flatten)
# we are only creating 2 topics
lda = LatentDirichletAllocation(n_components=10, random_state=0)
lda.fit(X)

LatentDirichletAllocation(random_state=0)

In [19]:
feature_names = count_vectorizer.get_feature_names_out()
for topic_id, topic in enumerate(lda.components_):
    print(f"Topic {topic_id}:")
    print(" ".join([feature_names[i] for i in topic.argsort()[:-6:-1]]))

Topic 0:
training may significant must depending
Topic 1:
model image different tasks data
Topic 2:
cloud computing big smbs data
Topic 3:
model generated images models data
Topic 4:
learning disease plant deep visual
Topic 5:
cloud computing based features services
Topic 6:
artificial intelligence series time banking
Topic 7:
classification image network architecture performance
Topic 8:
deep visual methods local network
Topic 9:
artificial intelligence human machine many


In [20]:
#Now let's see the probability of one of the sentence to belong to each topic

for name, paper_info in papers.items():
    abstract = paper_info['abstract']
    new_doc_bow = count_vectorizer.transform([abstract])
    # Calcular la distribución de temas para el nuevo documento
    topic_distribution = lda.transform(new_doc_bow)
    # print(f"Topic distribution for {name}:")
    paper_info['topics_prob'] = {}
    for topic_idx, topic_prob in enumerate(topic_distribution[0]):
        topic = lda.components_[topic_idx]
        topic  = " ".join([feature_names[i] for i in topic.argsort()[:-6:-1]])
        paper_info['topics_prob'][topic] =topic_prob
        # print(f"Topic {topic_idx}: {topic_prob:.4f}")


In [21]:
papers

{'010': {'abstract': 'Cloud computing is fast creating a revolution in the way information technology is used and procured by organizations and by individuals. In this article, we examine what cloud computing is and the importance of this new model of computing. We then examine non-military uses of cloud computing in governments across the globe, from the Unites States to Europe and Asia. Then, we look at the resource -people and computing -issues involved in shirting to cloud computing. The author then presents his six-step "Cloud Migration Strategy" for governmental agencies to shift to cloud computing. Finally, we look "over the horizon" to the implications for public sector organizations and the information technology community as the cloud computing revolution progresses.',
  'written_by': ['David Wyld'],
  'id': '34C4144FA95CCECB28B14B8159AB4498',
  'acknowledgePerson': ['C Johnson',
   'L Ulanoff',
   'C Anderson',
   '@ Knowledge',
   'D Wyld',
   'S Ferguson',
   'J Davis',
  

# SAVE DICT

In [22]:

# Guardar el diccionario en un archivo usando pickle
with open('papers_metadata.pickle', 'wb') as f:
    pickle.dump(papers, f)
